In [10]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [11]:
# Let's just make sure the model is loaded

!ollama pull qwen2.5
MODEL = "qwen2.5"

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest 
pulling 2bada8a74506... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 4.7 GB                         
pulling 66b9ea09bd5b... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   68 B                         
pulling eb4402837c78... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.5 KB                         
pulling 832dd9e00a68... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  11 KB                         
pulling 2f15b3218f05... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [12]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [13]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [14]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a summary of this website in markdown. \
If it includes news or announcements, then summarize these too.(only if they are present)\n\n"
    user_prompt += website.text
    return user_prompt

In [15]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [16]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return response['message']['content']

In [17]:
summarize("https://www.creative-tim.com/twcomponents/cheatsheet/")

'```markdown\n# Tailwind CSS Cheat Sheet Summary\n\nThis website serves as a comprehensive guide for developers using Tailwind CSS, providing quick access to commonly used utility classes and configurations. The content is organized into sections such as typography, layout, colors, shadows, and more, making it easy for users to find specific styles or settings.\n\n- **Typography**: Includes various font sizes, weights, line heights, and other typographic utilities.\n- **Layout**: Features columns, grid, flexbox, spacing, and responsive design utilities.\n- **Colors**: Lists predefined color palettes and utility classes for color manipulation.\n- **Shadows**: Provides options to add depth and dimension to elements through shadow effects.\n- **Other Sections**: Covers forms, animations, and more, with concise descriptions and examples.\n\nThe site is designed to be a one-stop reference tool, allowing developers to quickly apply Tailwind CSS styles without having to consult the official d

In [18]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [19]:
display_summary('https://edwarddonner.com')

# Summary

**Home Page:**
- The website is titled "Home - Edward Donner" and introduces Ed, who enjoys coding, experimenting with large language models (LLMs), DJing, and engaging in Hacker News.
- He co-founded Nebula.io, an AI company focusing on helping people discover their potential. The platform uses proprietary LLMs for talent discovery and has been patented.

**News/Announcements:**
- **January 23, 2025:** LLM Workshop – Hands-on with Agents
- **December 21, 2024:** Welcome, SuperDataScientists!
- **November 13, 2024:** Mastering AI and LLM Engineering – Resources
- **October 16, 2024:** From Software Engineer to AI Data Scientist – resources

**Connect Section:**
- Provides ways to get in touch with Ed, including email, LinkedIn, Twitter, Facebook, and a newsletter subscription form.

**Additional Content:**
- **Connect Four:** Describes it as an arena where LLMs compete against each other.
- **About Page:** Further details about Ed's background and Nebula.io.